In [1]:
from pathlib import Path
import csv
import os
from dotenv import load_dotenv
#from googleapiclient.discovery import build

In [6]:
# Construct the file path using Path from pathlib
watchLaterVideo = Path("Takeout") / "YouTube and YouTube Music" / "playlists" / "Watch later videos.csv"

# Check if the file exists
if not watchLaterVideo.exists():
    print(f"File not found: {watchLaterVideo}")
    exit(1)
else:
    print(f"File found: {watchLaterVideo}")
# Check if the file is a CSV file
if watchLaterVideo.suffix != '.csv':
    print(f"File is not a CSV: {watchLaterVideo}")
    exit(1)
else:
    print(f"File is a CSV: {watchLaterVideo}")


File not found: Takeout/YouTube and YouTube Music/playlists/Watch later videos.csv
File is a CSV: Takeout/YouTube and YouTube Music/playlists/Watch later videos.csv


In [7]:
from googleapiclient.discovery import build

load_dotenv()
GOOGLE_KEY = os.getenv("googleKey")
if not GOOGLE_KEY:
    print("Google API key not found in environment variables.")
    exit(1)
youtube = build("youtube", "v3", developerKey=GOOGLE_KEY)

In [8]:
def get_channel_info(video_id):
    try:
        # Get video details
        video_response = youtube.videos().list(
            part="snippet",
            id=video_id
        ).execute()

        if video_response['items']:
            return {
                "channelId": video_response["items"][0]["snippet"]["channelId"],
                "channelTitle": video_response["items"][0]["snippet"]["channelTitle"]
            }
        
    except Exception as e:
        print(f"API Error: {str(e)}")
    return None

In [ ]:
# Read the CSV file and retrieve "video ID"s and creation timestamps
video_ids = []
create_datatime = []
with open(watchLaterVideo, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    if 'Video ID' not in reader.fieldnames or 'Playlist video creation timestamp' not in reader.fieldnames:
        raise ValueError("CSV file does not contain required columns: 'Video ID' and 'Playlist video creation timestamp'")
    # Read each row and append the video ID and creation timestamp to the lists
    for row in reader:
        video_ids.append(row['Video ID'])
        create_datatime.append(row['Playlist video creation timestamp'])

# for first 10 video ID, get channel info and print it
video_ids = video_ids[:10]  # Limit to first 10 video IDs
# Retrieve channel info for each video ID
for idx, video_id in enumerate(video_ids):
    channel_info = get_channel_info(video_id)
    if channel_info: break
"""
    if channel_info:
        print(f"Video ID: {video_id}")
        print(f"Video Creation Date: {create_datatime[idx]}")      
        print(f"Channel ID: {channel_info['channelId']}")
        print(f"Channel Title: {channel_info['channelTitle']}")
        print()
    else:
        print(f"Could not retrieve info for Video ID: {video_id}")
"""
# store the channel info in a dictionary including the video_ids, create_datetime, channel_id, and channel_title
channel_info_dict = {}
for idx, video_id in enumerate(video_ids):
    channel_info = get_channel_info(video_id)
    if channel_info:
        channel_info_dict[video_id] = {
            "create_datetime": create_datatime[idx],
            "channelId": channel_info['channelId'],
            "channelTitle": channel_info['channelTitle']
        }
    else:
        print(f"Could not retrieve info for Video ID: {video_id}")
# Print the channel info dictionary
print("Channel Info Dictionary:")
for video_id, info in channel_info_dict.items():
    print(f"Video ID: {video_id}")
    print(f"Video Creation Date: {info['create_datetime']}")
    print(f"Channel ID: {info['channelId']}")
    print(f"Channel Title: {info['channelTitle']}")
    print()

# Define the output file path
output_file = Path("outputs") / "Watch later videos channel info.csv"

# Create parent directories if they don't exist
output_file.parent.mkdir(parents=True, exist_ok=True)

# Save the channel info dictionary to a new CSV file in the outputs directory
with open(output_file, mode='w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Video ID', 'Video Creation Date', 'Channel ID', 'Channel Title']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for video_id, info in channel_info_dict.items():
        writer.writerow({
            'Video ID': video_id,
            'Video Creation Date': info['create_datetime'],
            'Channel ID': info['channelId'],
            'Channel Title': info['channelTitle']
        })

Channel Info Dictionary:
Video ID: vgmqUhvQlww
Video Creation Date: 2025-05-03T03:56:05+00:00
Channel ID: UCPa1ivCpLcqP60OGqb9Yzqw
Channel Title: ElectrArc240

Video ID: YfhPJ8GxdgI
Video Creation Date: 2025-05-01T15:43:43+00:00
Channel ID: UCrZG5sGryxwgSDQSlHgmZTw
Channel Title: GadgetGang HK

Video ID: 2aGe20OuUAU
Video Creation Date: 2025-05-04T20:19:47+00:00
Channel ID: UCpGPq70Csdqhc1YCbJWQrJQ
Channel Title: The Honest Torus

Video ID: JJXEhmcQxwg
Video Creation Date: 2025-04-24T20:07:48+00:00
Channel ID: UCFGqfPwhO5N6LShbLmQT6xw
Channel Title: ZEISS Microscopy

Video ID: vg8yizC4W8E
Video Creation Date: 2025-05-02T00:31:03+00:00
Channel ID: UChVT0XLyNcKO-hPULCbqhrw
Channel Title: Hard派

Video ID: qqY8STwjpx4
Video Creation Date: 2025-04-19T20:07:08+00:00
Channel ID: UCH2t6jvINIOeYzBQR0iI5kw
Channel Title: 賭Sir【杜氏數學】HermanToMath

Video ID: ke7CaL-EXQg
Video Creation Date: 2025-04-18T08:25:32+00:00
Channel ID: UCKW_xzxlZlQQYm42HixNFEA
Channel Title: 0xmd

Video ID: bKBXqviTpaM
Vide